In [1]:
#dependency install
# %pip install pandas
# %pip install openpyxl

In [2]:
#imports
import csv
import pandas as pd
import hw0
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy

In [3]:
NAN = float("nan")

In [4]:
class HierarchicalClusteringHW1(hw0.HierarchicalClustering):
    def __init__(self, cluster_dist, silh_fun, return_distances=False, return_for_dendrogram=False):
        # the function that measures distances clusters (lists of data vectors)
        self.cluster_dist = cluster_dist

        # if the results of run() also needs to include distances;
        # if true, each joined pair in also described by a distance.
        self.return_distances = return_distances
        self.return_for_dendrogram = return_for_dendrogram
        self.silh_fun = silh_fun

    def run(self, data):
        """
        Performs hierarchical clustering until there is only a single cluster left
        and return a recursive structure of clusters.
        """

        # clusters stores current clustering. It starts as a list of lists
        # of single elements, but then evolves into lists like
        # [[["Albert"], [["Branka"], ["Cene"]]], [["Nika"], ["Polona"]]]
        clusters = [[name] for name in data.keys()]

        if self.return_for_dendrogram:
            linkages = []

        if self.return_for_dendrogram:
            avgSilhs = []
            avgSilhs.append((len(clusters), self.silh_fun(data, clusters)))

        while len(clusters) >= 2:
            first, second, distance = self.closest_clusters(data, clusters)
            # update the "clusters" variable

            if self.return_for_dendrogram:
                linkages.append([first, second, distance, 2])
            
            firstIndex, secondIndex = clusters.index(first), clusters.index(second)
            if self.return_distances:
                newClusters = [ [clusters[firstIndex], clusters[secondIndex], distance] ]
            else:
                newClusters = [ [clusters[firstIndex], clusters[secondIndex]] ]
            for cluster in clusters:
                if cluster == first or cluster == second: continue
                newClusters.append(cluster)
            clusters = newClusters

            if self.return_for_dendrogram:
                avgSilhs.append((len(clusters), self.silh_fun(data, clusters)))

        return (clusters,) if not self.return_for_dendrogram else (clusters, linkages, avgSilhs) 

In [5]:
def silhouette(el, clusters, data):
    """
    Za element el ob podanih podatkih data (slovar vektorjev) in skupinah
    (seznam seznamov nizov: ključev v slovarju data) vrni silhueto za element el.
    """

    if len(clusters) < 2: return NAN

    distanceFunc = hw0.euclidean_dist
    linkageFunc = hw0.average_linkage

    # find out what cluster contains the element
    belongId = None
    for i in range(len(clusters)):
        flatClusterI = hw0.flatten(clusters[i])
        if el in flatClusterI: 
            belongId = i
            break
    
    elData = data[el]

    # calc a
    aDistances = []
    flatClusterBI = hw0.flatten(clusters[belongId])
    for clusterEl in flatClusterBI:
        if clusterEl == el: continue
        ceData = data[clusterEl]
        aDistances.append(distanceFunc(elData, ceData))
    a = sum(aDistances)/len(aDistances) if len(aDistances)>0 else 0
    if a == 0: return 0 # thanks https://en.wikipedia.org/wiki/Silhouette_(clustering)#:~:text=Note%20that%20a(i)%20is%20not%20clearly%20defined%20for%20clusters%20with%20size%20%3D%201%2C%20in%20which%20case%20we%20set

    #calc b
    #   find closest cluster
    closestClusterId = None
    closestClusterDist = None
    for i in range(len(clusters)):
        if i == belongId: continue

        belongCluster = []
        for index in hw0.flatten(clusters[belongId]):
            belongCluster.append(data[index])

        compareCluster = []
        for index in hw0.flatten(clusters[i]):
            compareCluster.append(data[index])

        distance = linkageFunc(belongCluster, compareCluster, distanceFunc)
        if closestClusterDist is None or distance < closestClusterDist:
            closestClusterDist = distance
            closestClusterId = i
    
    bDistances = []
    for clusterEl in hw0.flatten(clusters[closestClusterId]):
        ceData = data[clusterEl]
        bDistances.append(distanceFunc(elData, ceData))
    b = sum(bDistances)/len(bDistances)

    s = (b - a) / max(a, b) if max(a, b) != 0 else NAN
    return s


def silhouette_average(data, clusters):
    """
    Za podane podatke (slovar vektorjev) in skupine (seznam seznamov nizov:
    ključev v slovarju data) vrni povprečno silhueto.
    """
    dataKeys = list(data.keys())
    silhs = []
    for key in dataKeys:
        calculatedSilh = silhouette(key, clusters, data)
        if calculatedSilh == calculatedSilh:
            silhs.append(silhouette(key, clusters, data))
    avgS = sum(silhs)/len(silhs) if len(silhs) > 0 else NAN
    
    return avgS

### Reading the data

In [6]:
# #read CSV
# rawDataHeader = []
# rawData = []
# with open("eurovision_song_contest_1957_2023.csv") as csvfile:
#     reader = csv.reader(csvfile, delimiter=",")
#     rawDataHeader = next(reader)
#     for row in reader:
#         rawData.append(row)

In [7]:
pathOriginalDataFile = "data/eurovision_song_contest_1957_2023.xlsx"
origDataFrame = pd.read_excel(pathOriginalDataFile)

In [8]:
origDataFrame.keys()[6]

'Points      '

In [9]:
#fixing this bug-inducing column/attribute name
oldkey = origDataFrame.keys()[6]
newkey = origDataFrame.keys()[6].rstrip()
print("\"{}\"".format(oldkey))
print("\"{}\"".format(newkey))
origDataFrame.rename(columns={oldkey : newkey}, inplace=True)

"Points      "
"Points"


In [10]:
origDataFrame.keys()[6]

'Points'

## Version 1

### Data preparation

Poskusimo tako, da oblikujemo tabelo, kjer so vrstice glasujoče države, stolpci pa države, za katere so glasovale. V celicah so števila točk glasovanja. Stolpci z glasovi so za vsako leto posebej, leta dodamo tabeli (širimo v desno).

In [ ]:
#drop entries that aren't for finals
prep1orig = origDataFrame[origDataFrame["(semi-) final"] == "f"]

In [ ]:
prep1years = prep1orig["Year"].unique()
prep1voters = prep1orig["From country"].unique()
prep1votedFor = prep1orig["To country"].unique()

prep1data = {}
for i in range(len(prep1voters)):
    voter = prep1voters[i]
    prep1data[voter] = []

for year in prep1years:
    yearSelection = prep1orig[prep1orig["Year"] == year]
    for voter in prep1voters:
        voterSelection = yearSelection[yearSelection["From country"] == voter]
        newVotes = []
        for votedFor in prep1votedFor:
            lineSelection = voterSelection[voterSelection["To country"] == votedFor]
            newVoteVal = NAN
            if len(lineSelection["Points"].values) > 0:
                newVoteVal = lineSelection["Points"].values[0]
            newVotes.append(newVoteVal)
        prep1data[voter] += newVotes

### Clustering

In [ ]:
#params
distance1 = hw0.euclidean_dist
linkage1 = hw0.single_linkage
distFun1 = lambda c1, c2: linkage1(c1, c2, distance1)
silhFun1 = silhouette_average

In [ ]:
hc1 = HierarchicalClusteringHW1(distFun1, silhFun1, return_distances=False, return_for_dendrogram=True)
clusters1 = hc1.run(prep1data)

### Visualization

In [ ]:
vis1codex = [[v] for v in prep1voters]
linkages1 = clusters1[1]
for line in linkages1:
    if line[0] not in vis1codex:
        vis1codex.append(line[0])
    if line[1] not in vis1codex:
        vis1codex.append(line[1])
    line[0] = vis1codex.index(line[0])
    line[1] = vis1codex.index(line[1])


In [ ]:
# Plot dendrogram
plt.figure(figsize=(20, 10))
dn = hierarchy.dendrogram(linkages1, labels=prep1voters)

# Show plot
plt.show()

In [ ]:
clusters1[2]

In [ ]:
vis1silGraphX, vis1silGraphY = zip(*clusters1[2])

plt.scatter(vis1silGraphX, vis1silGraphY)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette')
plt.grid(True)
plt.show()